# Data 

In [1]:
import torch
torch.set_default_dtype(torch.float64)

import torch.nn as nn
import torch.nn.functional as F

import torch.optim as optim

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [9]:
import os
!wget https://raw.githubusercontent.com/quaternion12345/lora/main/20210514.csv
os.rename(r'20210514.csv',r'rssi_data.csv')

--2021-05-21 06:28:07--  https://raw.githubusercontent.com/quaternion12345/lora/main/20210514.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 17334 (17K) [text/plain]
Saving to: ‘20210514.csv’

20210514.csv        100%[===================>]  16.93K  --.-KB/s    in 0s      

2021-05-21 06:28:07 (121 MB/s) - ‘20210514.csv’ saved [17334/17334]



In [10]:
rssi_data_df = pd.read_csv("rssi_data.csv",index_col=0)
rssi_data_df.head(n=10)

,rssi.242,rssi.94,rssi.h,rssi.245,outcome
1,-83,-101,-73,-74,1
2,-84,-99,-74,-75,1
3,-83,-101,-69,-74,1
4,-85,-100,-69,-75,1
5,-83,-103,-68,-73,1
6,-86,-93,-63,-81,1
7,-86,-92,-63,-80,1
8,-84,-94,-62,-81,1
9,-85,-92,-62,-81,1
10,-85,-94,-62,-81,1


In [ ]:
max(rssi_data_df["rssi.242"])

-60

In [ ]:
np.mean(rssi_data_df["rssi.242"])

-73.19654714475432

In [ ]:
rssi_data_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 753 entries, 1 to 753
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype
---  ------    --------------  -----
 0   rssi.242  753 non-null    int64
 1   rssi.94   753 non-null    int64
 2   rssi.h    753 non-null    int64
 3   rssi.245  753 non-null    int64
 4   outcome   753 non-null    int64
dtypes: int64(5)
memory usage: 35.3 KB


# Dense model

In [5]:
class DenseModel(nn.Module):
    def __init__(self, num_in):

        super(DenseModel, self).__init__()

        self.fc1 = nn.Linear(num_in, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 128)
        self.fc4 = nn.Linear(128, 64)
        self.ouput = nn.Linear(64, 11)
        #self.dropout = F.dropout(p=0.5)
        #self.dropout = nn.Dropout(p=0.5)

    def forward(self, x):
        x = F.gelu(self.fc1(x))
        x = F.gelu(self.fc2(x))
        x = F.gelu(self.fc3(x))
        x = F.gelu(self.fc4(x))

        #x = F.dropout(torch.sigmoid(self.fc3(x)), p=0.5)
        x = self.ouput(x)
        return x

dense = DenseModel(4)


In [ ]:
from sklearn.model_selection import KFold
def reset_weights(m):
  '''
    Try resetting model weights to avoid
    weight leakage.
  '''
  for layer in m.children():
   if hasattr(layer, 'reset_parameters'):
    print(f'Reset trainable parameters of layer = {layer}')
    layer.reset_parameters()

In [11]:
# 훈련, 테스트 데이터 shuffle & split
from sklearn.model_selection import train_test_split

TEST_PERCENTAGE = 0.2
train, test = train_test_split(rssi_data_df, test_size=TEST_PERCENTAGE)
print('train data length:',len(train))
print('test data length:',len(test))

train data length: 602
test data length: 151


In [ ]:
###################################
# You can skip this train process #
# Just download trained model     #
################################### 

In [ ]:
# Configuration options
k_folds = 5
num_epochs = 250
loss_function = nn.CrossEntropyLoss()

# For fold results
results = {}

# Set fixed random number seed
torch.manual_seed(42)

kfold = KFold(n_splits=k_folds, shuffle=True)
dataset = train.values # TensorFlow/Keras likes numpy/arrays rather than Dataframes

In [ ]:
# K-fold Cross Validation model evaluation
#network = DenseModel(4)

for fold, (train_ids, test_ids) in enumerate(kfold.split(dataset)):
  
  # Print
  print(f'FOLD {fold}')
  print('--------------------------------')
  
  # Sample elements randomly from a given list of ids, no replacement.
  train_subsampler = torch.utils.data.SubsetRandomSampler(train_ids)
  test_subsampler = torch.utils.data.SubsetRandomSampler(test_ids)
  
  # Define data loaders for training and testing data in this fold
  trainloader = torch.utils.data.DataLoader(
                    dataset, 
                    batch_size=10, sampler=train_subsampler)
  testloader = torch.utils.data.DataLoader(
                    dataset,
                    batch_size=10, sampler=test_subsampler)
  
  # Init the neural network
  network = DenseModel(4)
  network.apply(reset_weights)

  
  # Initialize optimizer
  optimizer = torch.optim.Adam(network.parameters(), lr=0.001)
  
  # Run the training loop for defined number of epochs
  for epoch in range(0, num_epochs):

    # Print epoch
    print(f'Starting epoch {epoch+1}')

    # Set current loss value
    current_loss = 0.0

    # Iterate over the DataLoader for training data
    for i, data in enumerate(trainloader, 0):

      # Get inputs
      inputs = data[:,:4]
      targets = data[:,4]
      # Zero the gradients
      optimizer.zero_grad()
      
      # Perform forward pass
      outputs = network(inputs.double())
      
      # Compute loss
      loss = loss_function(outputs, targets)
      
      # Perform backward pass
      loss.backward()
      
      # Perform optimization
      optimizer.step()
      
      # Print statistics
      current_loss += loss.item()
      if i % 10 == 0:
          print('Loss after mini-batch %5d: %.3f' %
                (i + 1, current_loss / 500))
          current_loss = 0.0
          
  # Process is complete.
  print('Training process has finished. Saving trained model.')

  # Print about testing
  print('Starting testing')

  # Saving the model
  save_path = f'./model-fold-{fold}.pth'
  torch.save(network.state_dict(), save_path)

  # Evaluation for this fold
  correct, total = 0, 0
  with torch.no_grad():

    # Iterate over the test data and generate predictions
    for i, data in enumerate(testloader, 0):

      # Get inputs
      inputs = data[:,:4]
      targets = data[:,4]

      # Generate outputs
      outputs = network(inputs.double())

      # Set total and correct
      _, predicted = torch.max(outputs.data, 1)
      total += targets.size(0)
      correct += (predicted == targets).sum().item()

    # Print accuracy
    print('Accuracy for fold %d: %d %%' % (fold, 100.0 * correct / total))
    print('--------------------------------')
    results[fold] = 100.0 * (correct / total)

In [ ]:
# Print fold results
print(f'K-FOLD CROSS VALIDATION RESULTS FOR {k_folds} FOLDS')
print('--------------------------------')
sum = 0.0
for key, value in results.items():
  print(f'Fold {key}: {value} %')
  sum += value
print(f'Average: {sum/len(results.items())} %')

K-FOLD CROSS VALIDATION RESULTS FOR 5 FOLDS
--------------------------------
Fold 0: 88.42975206611571 %
Fold 1: 89.25619834710744 %
Fold 2: 86.66666666666667 %
Fold 3: 88.33333333333333 %
Fold 4: 85.0 %
Average: 87.53719008264463 %


In [3]:
#######################################
# download best model from repository #
#######################################
!wget https://raw.githubusercontent.com/quaternion12345/lora/main/models/94p/model-fold-0.pth
!wget https://raw.githubusercontent.com/quaternion12345/lora/main/models/94p/model-fold-1.pth
!wget https://raw.githubusercontent.com/quaternion12345/lora/main/models/94p/model-fold-2.pth
!wget https://raw.githubusercontent.com/quaternion12345/lora/main/models/94p/model-fold-3.pth
!wget https://raw.githubusercontent.com/quaternion12345/lora/main/models/94p/model-fold-4.pth

--2021-05-21 06:27:10--  https://raw.githubusercontent.com/quaternion12345/lora/main/models/94p/model-fold-0.pth
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1409545 (1.3M) [application/octet-stream]
Saving to: ‘model-fold-0.pth’

model-fold-0.pth    100%[===================>]   1.34M  --.-KB/s    in 0.05s   

2021-05-21 06:27:10 (24.5 MB/s) - ‘model-fold-0.pth’ saved [1409545/1409545]

--2021-05-21 06:27:10--  https://raw.githubusercontent.com/quaternion12345/lora/main/models/94p/model-fold-1.pth
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response.

In [6]:
m0 = DenseModel(4)
m1 = DenseModel(4)
m2 = DenseModel(4)
m3 = DenseModel(4)
m4 = DenseModel(4)
m0.load_state_dict(torch.load('/content/model-fold-0.pth'))
m1.load_state_dict(torch.load('/content/model-fold-1.pth'))
m2.load_state_dict(torch.load('/content/model-fold-2.pth'))
m3.load_state_dict(torch.load('/content/model-fold-3.pth'))
m4.load_state_dict(torch.load('/content/model-fold-4.pth'))

m = [m0,m1,m2,m3,m4]

In [12]:
# 테스트 데이터셋 각 모델에 대해 돌려보기
out = []
for i in range(5):
  correct, total = 0, 0
  with torch.no_grad():
    # Get inputs
    inputs = torch.DoubleTensor(test.values[:,:4])
    targets = torch.DoubleTensor(test.values[:,4])

    # Generate outputs
    out.append(m[i](inputs.double()))

    # Set total and correct
    _, predicted = torch.max(out[i].data, 1)
    total += targets.size(0)
    correct += (predicted == targets).sum().item()

    # Print accuracy
    print('Test Accuracy for model %d: %d %%' % (i, 100.0 * correct / total))


Test Accuracy for model 0: 93 %
Test Accuracy for model 1: 92 %
Test Accuracy for model 2: 94 %
Test Accuracy for model 3: 90 %
Test Accuracy for model 4: 89 %


In [13]:
# 모델 앙상블
out_avg = (out[0] + out[1] + out[2] + out[3] + out[4]) / 5.
_, predicted = torch.max(out_avg.data, 1)

# accuracy 측정
targets = torch.DoubleTensor(test.values[:,4])

correct, total = 0, 0
total += targets.size(0)
correct += (predicted == targets).sum().item()

# 틀린 것들만 출력
print('Wrong Predictions: (Predicted / Ground Truth)')
for i in range(targets.size(0)):
  if predicted[i].item()!=int(targets[i].item()):
    print(predicted[i].item(), int(targets[i].item()))
    
# Print accuracy
print('--------------------------------')
print('Test Accuracy for model ensemble: %d %%' % (100.0 * correct / total))


Wrong Predictions: (Predicted / Ground Truth)
7 10
7 6
5 6
6 7
7 6
5 7
6 4
6 7
7 6
--------------------------------
Test Accuracy for model ensemble: 94 %
